In [1]:
import meshio
import numpy as np
import os.path as path
import stripy as stripy
from scipy import ndimage
from netCDF4 import Dataset

## Create mesh with stripy

In [2]:
# grid0 = stripy.spherical_meshes.icosahedral_mesh(include_face_points=False, refinement_levels=10)

# str_fmt = "{:25} {:9}"
# print(str_fmt.format('Number of points', grid0.npoints))
# print(str_fmt.format('Number of cells', grid0.simplices.shape[0]))

# mesh0 = meshio.Mesh(grid0.points, {'triangle': grid0.simplices})
# meshio.write("icosahedral-mesh.vtk", mesh0)

# Number of points           10485762
# Number of cells            20971520

In [3]:
# grid1 = stripy.spherical_meshes.triangulated_soccerball_mesh(refinement_levels=8)

# str_fmt = "{:25} {:9}"
# print(str_fmt.format('Number of points', grid1.npoints))
# print(str_fmt.format('Number of cells', grid1.simplices.shape[0]))

# mesh1 = meshio.Mesh(grid1.points, {'triangle': grid1.simplices})
# meshio.write("soccerball-mesh.vtk", mesh1)

In [5]:
grid2 = stripy.spherical_meshes.octahedral_mesh(include_face_points=False, refinement_levels=11)

str_fmt = "{:25} {:9}"
print(str_fmt.format('Number of points', grid2.npoints))
print(str_fmt.format('Number of cells', grid2.simplices.shape[0]))
# mesh2 = meshio.Mesh(grid2.points, {'triangle': grid2.simplices})
# meshio.write("octahedral-mesh.vtk", mesh2)

# Number of points           16777218
# Number of cells            33554432

Number of points            1048578
Number of cells             2097152


We also need to store each vertex neighbours... This is a pretty long process here it needs to be optimise I guess :-)

In [6]:
x = grid2.points[:,0]*6378137.
y = grid2.points[:,1]*6378137. 
h = grid2.points[:,2]*6378137. #+ elevations 

coords = np.vstack((x,y))
coords = np.vstack((coords,h)).T

In [7]:
ngbh = -np.ones((grid2.npoints,6))
for k in range(grid2.npoints):
    ids = grid2.identify_vertex_neighbours(k)
    ngbh[k,0:len(ids)] = ids
    if k%500000==0:
        print('Compute ',int(k*100./float(grid2.npoints)),'% of the neighborhood')

Compute  0.0 % of the neighborhood
Compute  47.683624871015795 % of the neighborhood
Compute  95.36724974203159 % of the neighborhood


In [8]:
np.savez_compressed('data/sphereHR/octohedral_lr', v=coords, c=grid2.simplices, n=ngbh.astype(int))

## Reading paleoelevation grid

In [9]:
elevfile = "data/scotese/60Ma.nc"
data = Dataset(elevfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

Define coordinates for interpolation on our mesh...

In [10]:
# Convert spherical mesh longitudes and latitudes to degrees
dlons = np.mod(np.degrees(grid2.lons)+180.0, 360.0)
dlats = np.mod(np.degrees(grid2.lats)+90, 180.0)

# Map mesh coordinates on ETOPO1 dataset
ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

## Performing interplolation...

In [11]:
elevations = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

In [13]:
x = grid2.points[:,0]*6378137.
y = grid2.points[:,1]*6378137. 
h = grid2.points[:,2]*6378137.+ elevations 

coords = np.vstack((x,y))
coords = np.vstack((coords,h)).T

## Save model input mesh

In [14]:
np.savez_compressed('data/scotese_cells_lr_60Ma', v=coords, c=grid2.simplices, n=ngbh.astype(int), z=elevations)

In [ ]:
mesh = meshio.Mesh(coords, {'triangle': grid.simplices}, {'Z':elevations})
meshio.write("data/60_11.vtk", mesh)

Loading a file and make interpolation

In [15]:
from time import clock
t0 = clock()
loaded = np.load("data/paleotopo/scotese_cells10Ma8.npz")
coords = loaded['v']
ngbhs = loaded['n']
cells = loaded['c']
elev = loaded['z']
print('loading time ',clock()-t0)

loading time  0.37361900000000503


In [ ]:
cells.dtype

In [16]:
radius = 6378137.0
lats = np.arcsin(coords[:,-1]/radius)
lons = np.arctan2(coords[:,1],coords[:,0]) 

dlons = np.mod(np.degrees(lons)+180.0, 360.0)
dlats = np.mod(np.degrees(lats)+90, 180.0)
   
elevfile = "data/scotese/60Ma.nc"
data = Dataset(elevfile, "r", format="NETCDF4")
img = np.fliplr(data['z'][:,:].T)

ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0
icoords = np.stack((ilons, ilats))
    
elev = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

In [17]:
meshAge = "data/lr_60Ma"
np.savez_compressed(meshAge, v=coords, c=cells, n=ngbhs, z=elev)

In [ ]:
mesh = meshio.Mesh(coords, {'triangle': cells}, {'Z':elev})
meshio.write("data/6km_60Ma.vtk", mesh)

Distances...

In [ ]:
import meshplex
Tmesh = meshplex.MeshTri(coords,loaded['c'])
Tmesh.create_edges()
edges_nodes = Tmesh.edges['nodes']

In [ ]:
e1 = edges_nodes[:,0]
e2 = edges_nodes[:,1]
dx = coords[e1,0]-coords[e2,0]
dy = coords[e1,1]-coords[e2,1]
dz = coords[e1,2]-coords[e2,2]
d2 = dx**2+dy**2+dz**2
dist = np.sqrt(d2)
print(dist.min(),dist.max(),np.mean(dist))